### Step 1: Read in data and drop all data for non-K12 persons 
Probably will have to do this for all analyses, unless we are looking also at family members. The data needs to include the "GRADEATT" and "SCHOOL" attributes. This portion starts with a df and creates df_k12 which includes only K-12 persons. 

See internet_access.ipynb for details on steps/the most comprehensive documentation out of all of these. 

In [1]:
import csv
import json
import numpy as np
import pandas as pd

df = pd.read_csv("../../../acs_data.csv")
df_students = df[df['SCHOOL'] == 2]
df_k12 = df_students[(df_students.GRADEATT != 1) & (df_students['GRADEATT'] != 6) & (df_students['GRADEATT'] != 7)]

### Step 2: Feature-specific analysis

In [4]:
# Drop rows w/o info on internet access
df_k12 = df_k12[df_k12['BEDROOMS'] != 0]

In [22]:
df_k12['few_rooms'] = 0
df_k12.loc[df_k12['BEDROOMS'] < df_k12['NUMPREC'] - 1,'few_rooms'] = 1

In [23]:
sum(df_k12['few_rooms'])/len(df_k12['few_rooms'])

0.1727517020101407

In [24]:
df_k12['few_rooms'] = df_k12['few_rooms'] * df_k12['PERWT']

In [25]:
sum(df_k12['PERWT'])

53592256.0

### Step 3: Output csv

In [27]:
# Get conversion from fips to state
with open("../FIPSconversion/fipsToState.json") as json_file:
    state_fips = json.load(json_file)

In [33]:
l = [] # list of dictionaries

for fips in state_fips:
    few_rooms = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'few_rooms'].sum()
    total = df_k12.loc[df_k12['STATEFIP'] == int(fips), 'PERWT'].sum()
    rate = int((few_rooms / total) * 100)
    state_name = state_fips[fips]
    l.append({'AREANM':state_name, 'AREACD':state_name, 'value':rate})


In [34]:
# output as CSV
out = pd.DataFrame(l)
out.to_csv('../bedrooms.csv',index = False)

In [ ]:
# output as JSON file
with open('../bedrooms.json', 'w') as fout:
    json.dump(l, fout)